In [1]:

import numpy as np
from utils import setup_args
import random
from data.fillers import SamplerFiller
args = setup_args()

args.env_name = "Pong-v0"
args.model_name = "vae"
args.frames_per_example = 10
args.batch_size = 256

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


In [2]:
sf = SamplerFiller(args)

In [3]:
fs = sf.fill(2000)

In [4]:
trans = fs.sample()
print(trans.xs.shape)
print(trans.actions.shape)

for trans in fs:
    print(trans.xs.shape)
    print(trans.actions.shape)

torch.Size([256, 10, 3, 128, 128])
torch.Size([256, 9])
torch.Size([256, 10, 3, 128, 128])
torch.Size([256, 9])
torch.Size([256, 10, 3, 128, 128])
torch.Size([256, 9])
torch.Size([256, 10, 3, 128, 128])
torch.Size([256, 9])
torch.Size([256, 10, 3, 128, 128])
torch.Size([256, 9])
torch.Size([256, 10, 3, 128, 128])
torch.Size([256, 9])
torch.Size([256, 10, 3, 128, 128])
torch.Size([256, 9])
torch.Size([256, 10, 3, 128, 128])
torch.Size([256, 9])
torch.Size([256, 10, 3, 128, 128])
torch.Size([256, 9])
torch.Size([147, 10, 3, 128, 128])
torch.Size([147, 9])
